In [ ]:
import pandas as pd
import numpy as np

In [ ]:
clientes_pd = pd.read_csv('in/Clientes.csv', sep=';', encoding='utf-8')
compra_pd = pd.read_csv('in/Compra.csv', encoding='utf-8')
gastos_pd = pd.read_csv('in/Gasto.csv', encoding='utf-8')
localidades_pd = pd.read_csv('in/Gasto.csv', encoding='utf-8')
proveedores_pd = pd.read_csv('in/Proveedores.csv')
sucursales_pd = pd.read_csv('in/Sucursales.csv', sep= ';',encoding='utf-8')
venta_pd = pd.read_csv('in/Sucursales.csv',sep=';',encoding='utf-8')




In [ ]:
clientes_pd.head()
compra_pd.head()
gastos_pd.head()
localidades_pd.head()
proveedores_pd.head()
sucursales_pd.head()
venta_pd.head()


,ID,Sucursal,Direccion,Localidad,Provincia,Latitud,Longitud
0,1,Cabildo,Av. Cabildo 1342,Ciudad de Buenos Aires,Ciudad de Buenos Aires,"-34,5678060","-58,4495720"
1,2,Palermo 1,Guatemala 5701,CABA,CABA,"-34,5790350","-58,4335660"
2,3,Palermo 2,Gral. Lucio Norberto Mansilla 2668,CABA,C deBuenos Aires,"-34,5959660","-58,4051500"
3,4,Corrientes,Av. Corrientes 2352,Ciudad de Buenos Aires,Bs As,"-34,6046850","-58,3987640"
4,5,Almagro,Venezuela 3650,Capital,Bs.As.,"-34,6173080","-58,4161790"


In [28]:
import os

in = 'in'

label_path = os.listdir(in)
labels = sorted(label_path)
print(f'Datasets Labels: {labels}')
num_class = len(labels)
image_files = [[os.path.join(in, class_name, x)
                for x in os.listdir(os.path.join(in, class_name))]
               for class_name in labels]

SyntaxError: invalid syntax (3624690819.py, line 3)

In [ ]:
import os

in = 'in'


labels = os.listdir(train_dir)

print(f'Datasets Labels: {labels}')
num_class = len(labels)
image_files = [[os.path.join(train_dir, class_name, x)
                for x in os.listdir(os.path.join(train_dir, class_name))]
               for class_name in labels]